# UNET MODEL
> Created May. 2025 <br>
> Nikhil Bisht<br>

# Definitions

In [8]:
# standard system modules
import os, sys
import h5py 
import argparse
# standard module for tabular data
import pandas as pd
import json

# standard module for array manipulation
import numpy as np
from itertools import permutations

# standard statistical module
import scipy.stats as st
from scipy import linalg
from scipy.stats import ks_2samp
import skimage as ski
from skimage.metrics import structural_similarity as ski_ssim, normalized_mutual_information as ski_nmi

# standard module for high-quality plots
from PIL import Image
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
mp.rcParams.update(mp.rcParamsDefault)
%matplotlib inline

# to plot pixelized images
import imageio.v3 as im

# standard research-level machine learning toolkit from Meta (FKA: FaceBook)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import tables
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import save_image
from modules import *

from tqdm import tqdm

# set a seed to ensure reproducibility
seed = 128
rnd  = np.random.RandomState(seed)

DATAFILE  = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/nb101_ML_dataset_AllData_AutoEnc.h5'
IMAGEINPUT = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/Unet/Images_Input_2D/'
IMAGEOUTPUT = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/Unet/Images_Output_2D/'
CORESET  = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/nb101_all_frames.h5'
DENSITYCUBES = '/data/cb1/nbisht/anvil_scratch/projects/128/B2/datasets/nb101_TimeseriesCubes_Density.npy'
MODELFILE = 'nnmodel.dict'

IMAGESIZE = 128
FRAMES = np.arange(0,90, 1)
FRAME_DIFF = 30
TEST_PERCENTAGE = 0.01

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cpu')

print(f'Available device: {str(DEVICE):4s}')

Available device: cpu 


# Functions

In [4]:
def img_transform(np_arr):
    return np.log10(np_arr)

def img_inverse_transform(np_arr):
    return np.power(10,np_arr)

def plot_test_img(input_arr, pred_output_arr, act_output_arr, label, test_loss, save_filename='xy_data.png', dont_show = False):

    fig = plt.figure(figsize=(12, 4))
    fig.tight_layout()
    
    ax  = fig.add_subplot(1, 3, 1)
    c = ax.pcolormesh(input_arr.T)
    fig.colorbar(c, ax=ax)
    ax.set_title('Input: '+str(label[0])+'_'+str(label[1])+'_'+str(label[2])+'\n Input Loss: '+'{:0.5f}'.format(test_loss[0]))
    ax.grid('both')

    ax2  = fig.add_subplot(1, 3, 2)
    c = ax2.pcolormesh(pred_output_arr.T)
    fig.colorbar(c, ax=ax2)
    ax2.set_title('Predicted Output '+str(label[0])+'_'+str(label[1])+'_'+str(label[2]))
    ax2.grid('both')

    ax3  = fig.add_subplot(1, 3, 3)
    c = ax3.pcolormesh(act_output_arr.T)
    fig.colorbar(c, ax=ax3)
    ax3.set_title('Actual Output '+str(label[0])+'_'+str(label[1])+'_'+str(label[2])+'\n Test Loss: '+'{:0.5f}'.format(test_loss[1]))
    ax3.grid('both')


    if save_filename:
        plt.savefig(save_filename)
    
    if dont_show==False:
        plt.show()
    plt.close()

def delete_folder_contents(folder):
    import shutil
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def get_images(input_dir):
    output_dic = {}
    for filename in os.listdir(input_dir):
        infile = open(os.path.join(input_dir, filename), 'r')
        i_file = json.load(infile)
        num = filename.split('_')[1]
        img_dic = i_file['img']
        label = i_file['label']
        output_dic[num] = {'img': img_dic, 'label': label}
        infile.close()
    return output_dic



# Load Images

In [5]:
input_image_dic = get_images(IMAGEINPUT)
output_image_dic = get_images(IMAGEOUTPUT)
input_image_arr = []
output_image_arr = []
label_arr = []
for key in input_image_dic.keys():
    input_image_arr.append(torch.tensor(input_image_dic[key]['img']))
    output_image_arr.append(torch.tensor(output_image_dic[key]['img']))
    label_arr.append(input_image_dic[key]['label'])
input_image_train, input_image_test, output_image_train, output_image_test, label_train, label_test = train_test_split(input_image_arr, output_image_arr, label_arr, 
                                                                                                                       shuffle = False, test_size=TEST_PERCENTAGE, random_state=seed)
    

In [ ]:
import torch.nn as nn

class UNet2D(nn.Module):
    def __init__(self, in_channels, out_channels, base_filters=32):
        super().__init__()
        # Encoder path
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_channels, base_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_filters, base_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.enc2 = nn.Sequential(
            nn.Conv2d(base_filters, base_filters*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_filters*2, base_filters*2, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        # (You can add more encoder levels here…)

        # Decoder path
        self.upconv2 = nn.ConvTranspose2d(base_filters*2, base_filters, kernel_size=2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(base_filters*2, base_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(base_filters, base_filters, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        # (Add more decoder levels to mirror the encoder…)

        # Final 1×1 conv to map to desired output channels
        self.outconv = nn.Conv2d(base_filters, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)                          # [B, F, H, W]
        e2 = self.enc2(self.pool(e1))              # [B, 2F, H/2, W/2]
        # (encode deeper levels…)

        # Decoder
        d2 = self.upconv2(e2)                      # [B, F, H, W]
        d2 = torch.cat([d2, e1], dim=1)            # skip connection
        d2 = self.dec2(d2)                         # [B, F, H, W]
        # (decode through further levels…)

        return self.outconv(d2)                    # [B, out_channels, H, W]


def train_unet(input_arr, output_arr, labels, args):
    X = torch.stack(input_arr, dim=0).float().unsqueeze(1)  # [N,1,128,128]
    Y = torch.stack(output_arr, dim=0).float().unsqueeze(1) # [N,1,128,128]

    dataset = TensorDataset(X, Y)
    loader  = DataLoader(dataset,
                         batch_size=args.batch_size,
                         shuffle=True,
                         num_workers=2,    # adjust based on your CPU
                         pin_memory=True)  # if using GPU

    # Initialize model, loss function, and optimizer
    model = UNet2D(in_channels=1, out_channels=1).to(DEVICE)
    # For trainable weights
    loss_w = torch.nn.Parameter(torch.ones(5, device=DEVICE), requires_grad=True)
    optimizer = optim.Adam(list(model.parameters()) + [loss_w], lr=args.lr)
    loss_history = []

    if args.loss_type == 'static':
        #Precompute initial-loss weights
        with torch.no_grad():
            x0 = input_arr[0].unsqueeze(0).unsqueeze(0).to(DEVICE)
            y0 = output_arr[0].unsqueeze(0).unsqueeze(0).to(DEVICE)
            pred0 = model(x0)
            L1_0   = F.l1_loss(pred0, y0).item()
            MSE_0  = F.mse_loss(pred0, y0).item()
            #NMI ranges from 1 (perfectly uncorrelated image values) to 2 (perfectly correlated image values, whether positively or negatively)
            NMI_0  = 2 - ski_nmi(y0.cpu().numpy()[0][0],pred0.cpu().numpy()[0][0])
            #SSIM ranges from 0 (no similarity) to 1 (perfect similarity), we are omitting cases where it is negative
            SSIM_0 = 1 - ski_ssim(y0.cpu().numpy()[0][0], pred0.cpu().numpy()[0][0], gradient=False, data_range=np.max(pred0.cpu().numpy()) - np.min(pred0.cpu().numpy()), channel_axis=None)
            MASS_0 = torch.abs((pred0.sum()-y0.sum())/y0.sum()).item()
            init_w = torch.tensor([1/L1_0, 1/abs(NMI_0), 1/MSE_0, 1/SSIM_0, 1/MASS_0], device=DEVICE)

    #Training
    for epoch in range(args.epochs):
        model.train()
        epoch_loss = 0.0
        # iterate over batches
        for inputs, targets in loader:
            x, y = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            pred = model(x)

            #Computing individual losses
            l1    = F.l1_loss(pred, y)
            mse   = F.mse_loss(pred, y)
            #the [0][0] is to get the first image in the batch assuming only 1 channel
            nmi   = 2 - ski_nmi(y.cpu().numpy()[0][0], pred.cpu().detach().numpy()[0][0])
            ssim_l=1 - ski_ssim(y.cpu().numpy()[0][0], pred.cpu().detach().numpy()[0][0], gradient=False, data_range=np.max(pred.cpu().detach().numpy()) - np.min(pred.cpu().detach().numpy()), channel_axis=None)
            mass  = torch.abs((pred.sum()-y.sum())/y.sum())

            #Stacking losses
            losses = torch.stack([l1, torch.tensor(nmi, device=DEVICE),  mse, torch.tensor(ssim_l, device=DEVICE), mass])

            if args.loss_type == 'static':
                #Case A: Static initial weights
                static_w = init_w / init_w.sum()
                total_loss = (static_w * losses).sum()
            else:
                #Case B: Trainable weights (softmax)
                w = torch.softmax(loss_w, dim=0)
                total_loss = (w * losses).sum()

            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()

        avg_loss = epoch_loss / len(dataset)          # true per-sample avg
        loss_history.append(avg_loss)
        print(f"Epoch {epoch+1}/{args.epochs} — Loss: {avg_loss:.4f}")

    #plotting the loss
    plt.plot(loss_history)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.savefig('./plots/'+args.run_name+'_training_loss.png')
    plt.close()

    # Save artifacts
    torch.save(model.state_dict(), MODELFILE)
    #torch.save(loss_w, "./plots/loss_weights.pt")


In [43]:
torch.stack(input_image_train, dim=0).float().unsqueeze(1).shape

torch.Size([2138, 1, 128, 128])

In [35]:
#launch training
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.batch_size = 1
args.image_size = IMAGESIZE
args.device = DEVICE
args.run_name = "Unet_2D"
args.epochs = 30
args.lr = 3e-4
args.loss_type = 'dynamic' # 'static' or 'dynamic'


train_unet(input_image_train, output_image_train, label_train, args)

TypeError: expected np.ndarray (got list)

# Documentation for different metrics
[scikit-image documentation](https://scikit-image.org/docs/0.25.x/api/skimage.metrics.html#module-skimage.metrics)

In [33]:
def plot_comparison(input_arr, pred_output_arr, act_output_arr, label, save_filename='xy_data.png', dont_show = False):
    input_str = 'INPUT IMAGE:\n'
    pred_str = 'PREDICTED IMAGE:\n'
    act_str = 'ACTUAL IMAGE:\n'
    # Calculate metrics
    input_MSE = ski.metrics.mean_squared_error(act_output_arr, input_arr)
    input_str+= 'MSE: '+'{:0.5f}'.format(input_MSE)+'\n'
    pred_MSE = ski.metrics.mean_squared_error(act_output_arr, pred_output_arr)
    pred_str+= 'MSE: '+'{:0.5f}'.format(pred_MSE)+'\n'
    actual_MSE = ski.metrics.mean_squared_error(act_output_arr, act_output_arr)
    act_str+= 'MSE: '+'{:0.5f}'.format(actual_MSE)+'\n'

    input_NMI = ski.metrics.normalized_mutual_information(act_output_arr, input_arr, bins=100)
    input_str+= 'NMI: '+'{:0.5f}'.format(input_NMI)+'\n'
    pred_NMI = ski.metrics.normalized_mutual_information(act_output_arr, pred_output_arr, bins=100)
    pred_str+= 'NMI: '+'{:0.5f}'.format(pred_NMI)+'\n'
    actual_NMI = ski.metrics.normalized_mutual_information(act_output_arr, act_output_arr, bins=100)
    act_str+= 'NMI: '+'{:0.5f}'.format(actual_NMI)+'\n'

    inversed_input_arr = img_inverse_transform(input_arr)
    inversed_pred_output_arr = img_inverse_transform(pred_output_arr)
    inversed_act_output_arr = img_inverse_transform(act_output_arr)
    input_PSNR = ski.metrics.peak_signal_noise_ratio(inversed_act_output_arr, inversed_input_arr, data_range=np.max(inversed_input_arr) - np.min(inversed_input_arr))
    input_str+= 'PSNR: '+'{:0.5f}'.format(input_PSNR)+'\n'
    pred_PSNR = ski.metrics.peak_signal_noise_ratio(inversed_act_output_arr, inversed_pred_output_arr, data_range=np.max(inversed_pred_output_arr) - np.min(inversed_pred_output_arr))
    pred_str+= 'PSNR: '+'{:0.5f}'.format(pred_PSNR)+'\n'
    actual_PSNR = ski.metrics.peak_signal_noise_ratio(inversed_act_output_arr, inversed_act_output_arr, data_range=np.max(inversed_act_output_arr) - np.min(inversed_act_output_arr))
    act_str+= 'PSNR: '+'{:0.5f}'.format(actual_PSNR)+'\n'

    input_SSI = ski.metrics.structural_similarity(act_output_arr, input_arr, gradient=False, data_range=np.max(input_arr) - np.min(input_arr), channel_axis=None)
    input_str+= 'SSI: '+'{:0.5f}'.format(input_SSI)+'\n'
    pred_SSI = ski.metrics.structural_similarity(act_output_arr, pred_output_arr, gradient=False, data_range=np.max(pred_output_arr) - np.min(pred_output_arr), channel_axis=None)
    pred_str+= 'SSI: '+'{:0.5f}'.format(pred_SSI)+'\n'
    actual_SSI = ski.metrics.structural_similarity(act_output_arr, act_output_arr, gradient=False, data_range=np.max(act_output_arr) - np.min(act_output_arr), channel_axis=None)
    act_str+= 'SSI: '+'{:0.5f}'.format(actual_SSI)+'\n'

    input_total_density = np.sum(input_arr)
    input_str+= 'Total Density: '+'{:0.5f}'.format(input_total_density)+'\n'
    pred_total_density = np.sum(pred_output_arr)
    pred_str+= 'Total Density: '+'{:0.5f}'.format(pred_total_density)+'\n'
    act_total_density = np.sum(act_output_arr)
    act_str+= 'Total Density: '+'{:0.5f}'.format(act_total_density)+'\n'

    fig = plt.figure(figsize=(12, 4))
    
    ax  = fig.add_subplot(1, 3, 1)
    c = ax.pcolormesh(input_arr.T)
    fig.colorbar(c, ax=ax)
    ax.set_title('Input: '+str(label[0])+'_'+str(label[1])+'_'+str(label[2]))
    ax.grid('both')
    ax.text(0.15, -0.25, input_str, fontsize=10, transform=plt.gcf().transFigure, bbox=dict(boxstyle="round", edgecolor='black', facecolor='white'))

    ax2  = fig.add_subplot(1, 3, 2)
    c = ax2.pcolormesh(pred_output_arr.T)
    fig.colorbar(c, ax=ax2)
    ax2.set_title('Predicted Output '+str(label[0])+'_'+str(label[1])+'_'+str(label[2]))
    ax2.grid('both')
    ax2.text(0.4, -0.25, pred_str, fontsize=10, transform=plt.gcf().transFigure, bbox=dict(boxstyle="round", edgecolor='black', facecolor='white'))

    ax3  = fig.add_subplot(1, 3, 3)
    c = ax3.pcolormesh(act_output_arr.T)
    fig.colorbar(c, ax=ax3)
    ax3.set_title('Actual Output '+str(label[0])+'_'+str(label[1])+'_'+str(label[2]))
    ax3.grid('both')
    ax3.text(0.7, -0.25, act_str, fontsize=10, transform=plt.gcf().transFigure, bbox=dict(boxstyle="round", edgecolor='black', facecolor='white'))


    if save_filename:
        plt.savefig(save_filename, bbox_inches='tight')
    
    if dont_show==False:
        plt.show()
    plt.close()

def compare_output(input_arr, output_arr, labels, args):
    # Load the trained model
    model = UNet2D(in_channels=1, out_channels=1).to(DEVICE)
    model.load_state_dict(torch.load(MODELFILE))
    model.eval()

    with torch.no_grad():
        for i in range(len(input_arr)):
            inputs = input_arr[i].unsqueeze(0).unsqueeze(0)
            targets = output_arr[i].unsqueeze(0).unsqueeze(0)
            label = labels[i]
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            # Save the output images
            output_image = outputs.cpu().numpy()
            plot_comparison(inputs[0][0].numpy(), output_image[0][0], targets[0][0].numpy(), label, save_filename='./2D_test_plots/img'+"_".join(str(x) for x in label)+'.png', dont_show = True)
    print("Testing completed and output images plotted.")

compare_output(input_image_test, output_image_test, label_test, args)

/tmp/ipykernel_1327156/2502573143.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODELFILE))
/home/nbisht/nikhil_20241011/lib/python3.

Testing completed and output images plotted.


## Notes

* Add info about what frame or time is being looked at
* look at the Unet structure and draw it out
* Also, is the MSE actually good? what does it represent in terms of value? if initial is set as final, what is the MSE then? as comparison
* add NMI, PSNR, SSI and total density to loss [here's a link](https://medium.com/@baicenxiao/strategies-for-balancing-multiple-loss-functions-in-deep-learning-e1a641e0bcc0)
